In [1]:
import string
import pandas as pd
import pickle
from pprint import pprint 
import nltk
from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer 
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import tokenize,simple_tokenize
from gensim import corpora
from gensim.models import TfidfModel
from gensim.test.utils import get_tmpfile
# Enable logging for gensim - optional
import logging
import os
from gensim.models.wrappers import LdaMallet
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.utils import shuffle
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KDTree
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import textwrap
from difflib import SequenceMatcher
from copy import deepcopy


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
os.environ['MALLET_HOME'] = '/Users/hei/documents/csfyp/topic_modeling/mallet-2.0.8'
mallet_path = './mallet-2.0.8/bin/mallet' # update this path

In [3]:
init_notebook_mode(connected=True)

# Config and Function Definition

In [2]:
load_from_disk_tok_corpus = True
load_from_disk_tok_rmcm_corpus = True
load_from_disk_del_tok_rmcm_corpus = True
load_from_disk_dict_corpus = True
load_from_disk_tfidf = True
load_from_disk_lda = True
load_from_disk_bigram = True
load_from_disk_doc2vec = True


load_shuf_tok_bi_stp_traintest_cvjd_dict_path = '../tokenized_corpus/shuf_tok_bi_stp_traintest_cvjd_dict.pickle'
load_shuf_tok_rmcm_bi_stp_traintest_cvjd_dict_path = '../tokenized_corpus/shuf_tok_rmcm_bi_stp_traintest_cvjd_dict.pickle'
load_shuf_del_tok_bi_stp_traintest_cvjd_dict_path = '../tokenized_corpus/shuf_del_tok_bi_stp_traintest_cvjd_dict.pickle'
load_shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict_path = '../tokenized_corpus/shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict.pickle'

load_train_data_path = "../trainData/train_datax42797.pickle"
load_test_data_path = "../testData/test_datax21949.pickle"

load_tfidfModel_path = "../tfidfModels/tfidfModel_traintest_bi_rmcm"
load_ldaModel_path = "../ldaModels/ldaModel_traintest_bi_rmcm"
load_mmcorpus_path = '../corpus/serialized_corpus_traintest_bi_rmcm.mm'
load_mmcorpus_dict_path = '../corpus/serialized_corpus_dict_traintest_bi_rmcm'
load_dictionary_path = '../corpus/dictionary_traintest_bi_rmcm'
load_bigramModel_path = '../bigramModels/traintest_bi'
load_doc2vecModel_path = '../doc2vecModels/traintest_bi_rmcm'

save_shuf_tok_bi_stp_traintest_cvjd_dict_path = '../tokenized_corpus/shuf_tok_bi_stp_traintest_cvjd_dict.pickle'
save_shuf_tok_rmcm_bi_stp_traintest_cvjd_dict_path = '../tokenized_corpus/shuf_tok_rmcm_bi_stp_traintest_cvjd_dict.pickle'
save_shuf_del_tok_bi_stp_traintest_cvjd_dict_path = '../tokenized_corpus/shuf_del_tok_bi_stp_traintest_cvjd_dict.pickle'
save_shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict_path = '../tokenized_corpus/shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict.pickle'

save_tfidfModel_path = "../tfidfModels/tfidfModel_traintest_bi_rmcm"
save_ldaModel_path = "../ldaModels/ldaModel_traintest_bi_rmcm"
save_mmcorpus_path = '../corpus/serialized_corpus_traintest_bi_rmcm.mm'
save_mmcorpus_dict_path = '../corpus/serialized_corpus_dict_traintest_bi_rmcm'
save_dictionary_path = '../corpus/dictionary_traintest_bi_rmcm'
save_bigramModel_path = '../bigramModels/traintest_bi'
save_doc2vecModel_path = '../doc2vecModels/traintest_bi_rmcm'

use_nltk_tokenize = True

In [5]:
gensim_stopword_list = set("""
a about above across after afterwards again against all almost alone along already also although always am among amongst amoungst amount an and another any anyhow anyone anything anyway anywhere are around as at back be
became because become becomes becoming been before beforehand behind being below beside besides between beyond bill both bottom but by call can
cannot cant co computer con could couldnt cry de describe
detail did didn do does doesn doing don done down due during
each eg eight either eleven else elsewhere empty enough etc even ever every everyone everything everywhere except few fifteen
fify fill find fire first five for former formerly forty found four from front full further get give go
had has hasnt have he hence her here hereafter hereby herein hereupon hers herself him himself his how however hundred i ie
if in inc indeed interest into is it its itself keep last latter latterly least less ltd
just
kg km
made make many may me meanwhile might mill mine more moreover most mostly move much must my myself name namely
neither never nevertheless next nine no nobody none noone nor not nothing now nowhere of off
often on once one only onto or other others otherwise our ours ourselves out over own part per
perhaps please put rather re
quite
rather really regarding
same say see seem seemed seeming seems serious several she should show side since sincere six sixty so some somehow someone something sometime sometimes somewhere still such take ten
than that the their them themselves then thence there thereafter thereby therefore therein thereupon these they thick thin third this those though three through throughout thru thus to together too top toward towards twelve twenty two un under
until up unless upon us used using
various very very via
was we well were what whatever when whence whenever where whereafter whereas whereby wherein whereupon wherever whether which while whither who whoever whole whom whose why will with within without would yet you
your yours yourself yourselves
""".split())
jd_stopword_list = set(['experience','gender','sex','race','religion','age','sexual_orientation','marital_status','national_origin','sexual_orientation',
                       'gender_identity','ability','experience','protected_veteran','equal_employment','employment',
                       'employee','employed','employer','employment','employees','employers','equal_opportunity',
                       'employer_minority', 'female_disability', 'veteran','race_color', 'religion_creed',
                        'sex_sexual', 'orientation_gender','jobs','job','years','year','qualifications','qualification',
                        'required','require','work','worked','works','working','position','positions','education','educations','preferred',
                        'prefer','including','include','candidate','candidates','strong','knowledge','description',
                        'descriptions','disability','consideration','consider','considered','will','ll',
                        'proud', 'qualified_applicants', 'receive_consideration', 'employment_without',
                        'regard', 'expression', 'genetics_disability', 'veteran_status','fair_employment',
                        'age_race', 'creed_color', 'ancestry_marital', 'status_affectional', 'expression',
                        'disability_nationality', 'regarded', 'qualified_applicants', 'considered_without'
                       ])
nltk_stopword_list =  set(stopwords.words('english'))
my_stopword_list = nltk_stopword_list.union(jd_stopword_list).union(gensim_stopword_list)


# Load data from disk

In [ ]:
with open(load_shuf_traintest_cvjd_dict_path, "rb") as f:
    shuf_traintest_cvjd_dict = pickle.load(f)

In [ ]:
shuf_traintest_keys = []
shuf_traintest_jts = []
shuf_traintest_cvjds = []
shuf_traintest_skills = []
for key in shuf_traintest_cvjd_dict:
    shuf_traintest_keys.append(key)
    shuf_traintest_jts.append(shuf_traintest_cvjd_dict[key][0])
    shuf_traintest_cvjds.append(shuf_traintest_cvjd_dict[key][1])
    shuf_traintest_skills.append(shuf_traintest_cvjd_dict[key][2])
print (len(shuf_traintest_keys) == len(shuf_traintest_jts) and
       len(shuf_traintest_jts) == len(shuf_traintest_cvjds) and
       len(shuf_traintest_cvjds) == len(shuf_traintest_skills)) 

# Preprocessing

0. tokenize
1. bigram training and extraction
2. trigram training and extraction (optional)
3. remove stopwords (stopphrase)

In [6]:
def train_bigram(tok_traintest_cvjds):
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(tok_traintest_cvjds, min_count=64, threshold=64) # higher threshold fewer phrases.
    #trigram = gensim.models.Phrases(bigram[tok_stp_traintest_jds], threshold=50)  
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    #trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod


In [7]:
def ispunct(s):
    alpha = [char for char in s if char.isalnum()]
    if alpha == []:
        return True
    else:
        return False

def rm_stopwords(tok_doc):
    return [term for term in tok_doc if not term in my_stopword_list]

def my_tokenize(documents):
    #tokenize
    lowered_tok_documents = [nltk.wordpunct_tokenize(doc.lower()) for doc in documents]
    tok_documents = [[term for term in tok_doc if len(term)>1 and
                      not term.isnumeric() and not ispunct(term)]
                     for tok_doc in lowered_tok_documents]
    print ("tokenized!")
    #bigram
    if load_from_disk_bigram:
        bigram_model = gensim.models.phrases.Phraser.load(load_bigramModel_path)    
        print ("bigram model loaded!")
    else:
        bigram_model = train_bigram(tok_documents)
        with open(save_bigramModel_path,'wb') as f:
            bigram_model.save(f)
        print ("bigram model trained!")
    
    tok_bi_documents = [bigram_model[tok_doc] for tok_doc in tok_documents]
    
    print ("bigram model applied!")
    #stopword removal
    tok_bi_stp_documents =  [rm_stopwords(tok_doc) for tok_doc in tok_bi_documents]
    print ("stopword removed!")
    
    return tok_bi_stp_documents

In [57]:
print (test_jdX[1])

NameError: name 'test_jdX' is not defined

In [ ]:
idx2tag = dict([i for i in enumerate(shuf_traintest_markers)])
tag2idx = {idx2tag[key]:key for key in idx2tag}

# Tokenize

In [8]:
if load_from_disk_tok_corpus == True:
    with open(load_shuf_tok_bi_stp_traintest_cvjd_dict_path,"rb") as f:
        shuf_tok_bi_stp_traintest_cvjd_dict = pickle.load(f)
else:
    shuf_tok_bi_stp_traintest_cvjds = my_tokenize(shuf_traintest_cvjds)
    shuf_tok_bi_stp_traintest_cvjd_dict = dict(zip(shuf_traintest_keys,zip(shuf_traintest_jts,shuf_tok_bi_stp_traintest_cvjds,shuf_traintest_skills,shuf_traintest_cvjds)))

    with open(save_shuf_tok_bi_stp_traintest_cvjd_dict_path,"wb") as f:
        pickle.dump(shuf_tok_bi_stp_traintest_cvjd_dict,f)
    

In [9]:

print (shuf_tok_bi_stp_traintest_cvjd_dict['cvX_4297'])

('Senior IT Recruiter/Account Management - FTE', ['million', 'consulting', 'staffing', 'outsourcing', 'company', 'widely', 'recognized', 'industry', 'leading', 'innovative', 'firm', 'doherty', 'minnesota', 'largest', 'staffing', 'firm', 'minnesota', '2nd', 'largest', 'woman', 'owned', 'business', 'nation', '10th', 'largest', 'diversity', 'staffing', 'firm', 'search', 'direct', 'hire', 'contract', 'consulting', 'services', 'ats', 'avionte', 'provide', 'cycle', 'recruiting', 'services', 'recruitment', 'strategy', 'sourcing', 'interviewing', 'selection', 'process', 'offer', 'management', 'process', 'documentation', 'compliance', 'management', 'status', 'communication', 'hiring', 'managers', 'recruit', 'professional', 'developers', 'engineers', 'architects', 'bi', 'meet', 'client', 'needs', 'phone', 'screen', 'person', 'skype', 'interviews', 'potential', 'determine', 'represented', 'rht', 'utilize', 'search_engines', 'dice', 'monster', 'career', 'builder', 'linkedin', 'source', 'track', 'a

In [10]:
nltk.wordpunct_tokenize("I love (luv) aws(Amzaon Web.Service) JSP.NET Java/J2Ee")
nltk.word_tokenize("I love (luv) aws(Amzaon Web.Service) JSP.NET Java/J2Ee")

['I',
 'love',
 '(',
 'luv',
 ')',
 'aws',
 '(',
 'Amzaon',
 'Web.Service',
 ')',
 'JSP.NET',
 'Java/J2Ee']

# Remove Longest common substring

In [11]:
def longestSubstring(str1,str2): 
    seqMatch = SequenceMatcher(None,str1,str2) 
    # output will be like Match(a=0, b=0, size=5) 
    match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
    # print longest substring 
    return match.size,match.a,match.b

def rm_common_company_substring(cvjd_dict,min_threshold,max_threshold):
    #common substring removal
    cnt = 0
    jd_dict = {key: cvjd_dict[key] for key in cvjd_dict if key.startswith("jdX")}
    new_jd_dict = deepcopy(jd_dict)
    new_jd_deletedLen_dict = {key: 0 for key in new_jd_dict}
    new_jd_toBeDel_dict = {key: None for key in new_jd_dict}
    cm_substring_dict = {}
    for key_a in jd_dict:
        company_a = jd_dict[key_a][0].split(" | ")[1]
        tok_doc_a = jd_dict[key_a][1]
        for key_b in jd_dict:
            if(key_a != key_b):
                company_b = jd_dict[key_b][0].split(" | ")[1]
                tok_doc_b = jd_dict[key_b][1]
                if(company_a == company_b):
                    lcs = longestSubstring(tok_doc_a,tok_doc_b)
                    if (lcs[0] >= min_threshold and lcs[0] < max_threshold):
                        if(new_jd_deletedLen_dict[key_a] < lcs[0]):
                            new_jd_toBeDel_dict[key_a] = (lcs[1],lcs[1]+lcs[0])
                            new_jd_deletedLen_dict[key_a] = lcs[0]
                        if(new_jd_deletedLen_dict[key_b] < lcs[0]):
                            new_jd_toBeDel_dict[key_b] = (lcs[2],lcs[2]+lcs[0])
                            new_jd_deletedLen_dict[key_b] = lcs[0]
#                         print (new_jd_deletedLen_dict[key_a])
#                         print (new_jd_deletedLen_dict[key_b])
#                         print (new_jd_toBeDel_dict[key_a])
#                         print (new_jd_toBeDel_dict[key_b])
#         if (new_jd_deletedLen_dict[key_a]>200):
#             print (new_jd_deletedLen_dict[key_a])
#             print (tok_doc_a[new_jd_toBeDel_dict[key_a][0]:new_jd_toBeDel_dict[key_a][1]])
#             print ("\n")
                        #return
                        #print ("\n")
                        #print (jd_dict[key_a])
                        #print ("\n\n\n")
                        #print (jd_dict[key_b])
                        #print ("\n\n\n")
        cnt+=1
        if cnt%(210) == 0:
            print (cnt/(21000/100))
    print ("marked! Delete now!")
    for key in new_jd_dict:
        if(new_jd_deletedLen_dict[key] > 0):
            del new_jd_dict[key][1][new_jd_toBeDel_dict[key][0]:new_jd_toBeDel_dict[key][1]]
    
    return new_jd_dict

In [12]:
if load_from_disk_tok_rmcm_corpus == True:
    with open(load_shuf_tok_rmcm_bi_stp_traintest_cvjd_dict_path,"rb") as f:
        shuf_tok_rmcm_bi_stp_traintest_cvjd_dict = pickle.load(f)
else:
    new_jd_dict = rm_common_company_substring(shuf_tok_bi_stp_traintest_cvjd_dict,20,200)
    shuf_tok_rmcm_bi_stp_traintest_cvjd_dict = deepcopy(shuf_tok_bi_stp_traintest_cvjd_dict)
    shuf_tok_rmcm_bi_stp_traintest_cvjd_dict.update(new_jd_dict)
    with open(save_shuf_tok_rmcm_bi_stp_traintest_cvjd_dict_path,"wb") as f:
        pickle.dump(shuf_tok_rmcm_bi_stp_traintest_cvjd_dict,f)
    

# Further stopword removal

In [13]:
tar = 'jdX_1414'
print (shuf_tok_bi_stp_traintest_cvjd_dict[tar][1])
print ("\n")
print (shuf_tok_rmcm_bi_stp_traintest_cvjd_dict[tar][1])
print ("\n")
print (shuf_tok_rmcm_bi_stp2_traintest_cvjd_dict[tar][1])
print ("\n")
print (shuf_traintest_cvjd_dict[tar][1])

['dawn', 'new', 'space', 'lockheed_martin', 'pioneer', 'partner_innovator', 'builder', 'amazing_men', 'women', 'mission', 'make_difference', 'world', 'day', 'use', 'unique', 'skills', 'experiences', 'create', 'design', 'build', 'solutions', 'worlds_hardest', 'engineering', 'problems', 'culture_encourages', 'dream_big', 'perform', 'excellence', 'create', 'incredible', 'products', 'provide', 'resources', 'inspiration', 'focus', 'passion', 'courage', 'dream_big', 'want', 'build', 'better_tomorrow', 'looking', 'career', 'purpose', 'challenges', 'mind', 'grows', 'skills', 'contributes', 'nation_most', 'critical', 'missions', 'pride', 'solving', 'problems', 'think', 'impossible', 'answered', 'yes', 'questions', 'like', 'join', 'lockheed_martin', 'space', 'software', 'systems', 'cyber', 'operations', 's2co', 'team', 's2co', 'develops', 'delivers', 'operates', 'affordable', 'engineering', 'sustainment', 'solutions', 'commercial', 'military', 'intelligence', 'users', 'globe', 'team', 'develops'

NameError: name 'shuf_tok_rmcm_bi_stp2_traintest_cvjd_dict' is not defined

In [ ]:
#shuf_tok_rmcm_bi_stp_traintest_cvjd_dict_backup = shuf_tok_rmcm_bi_stp_traintest_cvjd_dict.copy()
shuf_tok_rmcm_bi_stp_traintest_cvjd_dict = {key: (shuf_tok_rmcm_bi_stp_traintest_cvjd_dict[key][0],
                                                  [tok for tok in shuf_tok_rmcm_bi_stp_traintest_cvjd_dict[key][1]
                                                    if not tok in my_stopword_list],
                                                  shuf_tok_rmcm_bi_stp_traintest_cvjd_dict[key][2],
                                                  shuf_tok_rmcm_bi_stp_traintest_cvjd_dict[key][3]
                                                 )
                                            for key in shuf_tok_rmcm_bi_stp_traintest_cvjd_dict}
       
       

In [ ]:
shuf_tok_rmcm_bi_stp_traintest_cvjd_dict['cvX_1']

# Remove documents with 0 tokens

In [12]:
if load_from_disk_del_tok_rmcm_corpus == True:
    with open(load_shuf_del_tok_bi_stp_traintest_cvjd_dict_path,"rb") as f:
        shuf_del_tok_bi_stp_traintest_cvjd_dict = pickle.load(f)
    with open(load_shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict_path,"rb") as f:
        shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict = pickle.load(f)
else:
    cnt = 0
    shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict = deepcopy(shuf_tok_rmcm_bi_stp_traintest_cvjd_dict)
    shuf_del_tok_bi_stp_traintest_cvjd_dict = deepcopy(shuf_tok_bi_stp_traintest_cvjd_dict)
    for key in shuf_tok_rmcm_bi_stp_traintest_cvjd_dict:
        t = shuf_tok_rmcm_bi_stp_traintest_cvjd_dict[key]
        if len(t[1])==0:
            del shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[key]
            del shuf_del_tok_bi_stp_traintest_cvjd_dict[key]
            cnt += 1
    print (cnt)
    with open(save_shuf_del_tok_bi_stp_traintest_cvjd_dict_path,"wb") as f:
        pickle.dump(shuf_del_tok_bi_stp_traintest_cvjd_dict,f)
    with open(save_shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict_path,"wb") as f:
        pickle.dump(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict,f)

In [6]:
shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict['cvY_343']

('Care Coordinator',
 ['provide',
  'care',
  'coordination',
  'new',
  'established',
  'patients',
  'performing',
  'tasks',
  'needed',
  'enable',
  'quality',
  'care',
  'requesting',
  'records',
  'referring',
  'specialists',
  'scheduling',
  'rescheduling',
  'house',
  'appointments',
  'communicating',
  'steps',
  'care',
  'patient',
  'cell',
  'phone',
  'technology',
  'maintaining',
  'open',
  'line',
  'communication',
  'event',
  'questions',
  'concerns',
  'prescriptions',
  'local',
  'pharmacies',
  'perform',
  'follow',
  'ups',
  'check',
  'symptoms',
  'long_term',
  'health',
  'goals',
  'escalating',
  'necessary',
  'medical',
  'questions',
  'appropriate',
  'staff',
  'np',
  'mds',
  'board'],
 None,
 'Provide Care Coordination for new and established patients. Performing all tasks needed to enable quality of care from requesting records, referring to specialists, scheduling and rescheduling for in house appointments. Communicating all steps in

In [ ]:
for key in set(shuf_tok_rmcm_bi_stp_traintest_cvjd_dict.keys()).difference(set(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict)):
    print (shuf_tok_bi_stp_traintest_cvjd_dict[key])
    

# Prepare corpus and dictionary

In [8]:
if load_from_disk_dict_corpus:
    dictionary = gensim.corpora.Dictionary.load_from_text(load_dictionary_path)
    mmcorpus = gensim.corpora.MmCorpus(load_mmcorpus_path)
    with open(load_mmcorpus_dict_path, 'rb') as f:
        mmcorpus_dict = pickle.load(f)
    
else:
    jts, tok_docs, skills, docs = zip(*list(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict.values()))
    
    dictionary = corpora.Dictionary(tok_docs)
    dictionary.save_as_text(save_dictionary_path)

    mmcorpus = [dictionary.doc2bow(doc) for doc in tok_docs]
    corpora.MmCorpus.serialize(save_mmcorpus_path, mmcorpus)  # store to disk, for later use
    mmcorpus_dict = dict(zip(list(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict.keys()),mmcorpus))
    with open(save_mmcorpus_dict_path,'wb') as f:
        pickle.dump(mmcorpus_dict,f)
    #pprint(corpus)

# Prepare TF-IDF

In [41]:
if load_from_disk_tfidf:
    tfidfModel = TfidfModel.load(load_tfidfModel_path)
else:
    tfidfModel = TfidfModel(mmcorpus)
    tfidfModel.save(save_tfidfModel_path)

In [ ]:
tfidfModel[mmcorpus_dict['cvX_200']]

# Prepare LDA

In [42]:
if load_from_disk_lda:
    ldaModel = gensim.models.ldamodel.LdaModel.load(load_ldaModel_path)
else:
    %time
    ldaModel = gensim.models.ldamodel.LdaModel(corpus=mmcorpus,
                                               id2word=dictionary,
                                               num_topics=30,
                                               update_every=1,
                                               passes=1,
                                               chunksize=1024,
                                               alpha='auto',
                                               eta=None, decay=0.5,
                                               offset=1.0, eval_every=10,
                                               iterations=50,
                                               gamma_threshold=0.001,
                                               minimum_probability=0.01,
                                               random_state=None,
                                               ns_conf=None,
                                               minimum_phi_value=0.01,
                                               per_word_topics=True,
                                               callbacks=None)
    # extract 100 LDA topics, using 1 pass and updating once every 1 chunk (10,000 documents)
    #using serial LDA version on this node
    #running online LDA training, 100 topics, 1 passes over the supplied corpus of 3931787 documents, updating model once every 10000 documents
    ldaModel.save(save_ldaModel_path)
    
    

# LDA results

In [129]:
def my_get_term_topics(ldaModel,term):
    term_topics_df = pd.DataFrame(columns=["word_{}".format(i) for i in range(0,10)]+["weight"])
    for topic in ldaModel.get_term_topics(dictionary.token2id[term],0.000):
        row_dict = {"word_{}".format(i): [ldaModel.show_topic(topic[0])[i][0]] for i in range(0,10)}
        row_dict.update({"weight":[topic[1]]})
        tmp_df = pd.DataFrame(row_dict)
        term_topics_df = term_topics_df.append(tmp_df)
    sorted_term_topics_df = term_topics_df.sort_values(by=['weight'],ascending=False)
    return sorted_term_topics_df

def my_get_document_topics(ldaModel,mmcorpus_dict,key):
#     text_df = pd.DataFrame([{"text":stp_train_jds[bow_doc_idx]}],columns=["text"])
#     with pd.option_context('display.max_colwidth', 100):
#         pd.options.display.expand_frame_repr=True
    document_topic_df = pd.DataFrame(columns=["word_{}".format(i) for i in range(0,10)]+["weight"])
    for topic in ldaModel.get_document_topics(mmcorpus_dict[key],minimum_probability=0.000, minimum_phi_value=None, per_word_topics=False):
        #print (topic)
        row_dict = {"word_{}".format(i): [ldaModel.show_topic(topic[0])[i][0]] for i in range(0,10)}
        #print ()
        row_dict.update({"weight":[topic[1]]})
        tmp_df = pd.DataFrame(row_dict)
        document_topic_df = document_topic_df.append(tmp_df)
    sorted_document_topic_df = document_topic_df.sort_values(by=['weight'],ascending=False)
    return sorted_document_topic_df

def my_get_lda_doc_vec(ldaModel,mmcorpus_dict,key,ignore_topics=[]):
    vec = [0 for i in range(30)]
    for (topic_no,weight) in ldaModel.get_document_topics(mmcorpus_dict[key],minimum_probability=0.000,
                                                          minimum_phi_value=None, per_word_topics=False):
        if not topic_no in ignore_topics:
            vec[topic_no] = weight
    return vec

def my_get_top_topics(ldaModel):
    top_topic_df = pd.DataFrame(columns=["word_{}".format(i) for i in range(0,30)]+["coherence score"])
    for topic in ldaModel.top_topics(corpus=mmcorpus, texts=None, dictionary=dictionary, window_size=None, coherence='u_mass', topn=30, processes=-1):
        row_dict = {"word_{}".format(i): topic[0][i][1] for i in range(0,30)}
        #print (topic)
        #pprint (row_dict)
        row_dict.update({"coherence score":[topic[1]]})
        tmp_df = pd.DataFrame(row_dict)
        top_topic_df = top_topic_df.append(tmp_df)
    return top_topic_df


In [30]:
my_get_top_topics(ldaModel)

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,...,word_21,word_22,word_23,word_24,word_25,word_26,word_27,word_28,word_29,coherence score
0,services,enterprise,development,solutions,infrastructure,technology,cloud,systems,architecture,management,...,application,lead,platform,support,engineering,based,security,delivery,business,-0.811097
0,web,application,data,developed,server,sql,net,javascript,asp_net,services,...,ui,database,pages,designed,framework,mvc,applications,environment,custom,-0.959353
0,project,business,requirements,team,development,process,user,agile,design,functional,...,new,release,document,projects,lead,software,documentation,meetings,users,-0.967748
0,management,provide,support,ensure,manage,develop,business,processes,maintain,service,...,internal,related,operations,requirements,standards,information,policies,activities,perform,-0.996901
0,project,management,managed,projects,program,team,process,developed,provided,budget,...,teams,plans,reporting,schedule,multiple,implementation,internal,time,contract,-1.008459
0,java,application,developed,web,spring,services,involved,framework,data,xml,...,database,components,designed,struts,sql,based,code,junit,service,-1.035806
0,support,windows,server,software,network,users,user,servers,desktop,hardware,...,end,setup,systems,managed,installed,administration,client,pc,environment,-1.037761
0,data,reports,application,developed,applications,oracle,systems,project,new,support,...,designed,created,processing,accounting,business,programs,implementation,production,custom,-1.041885
0,software,systems,hardware,equipment,network,maintenance,support,installation,troubleshooting,problems,...,configuration,networks,new,site,data,procedures,devices,issues,user,-1.046228
0,servers,server,linux,environment,storage,windows,administration,configuration,systems,unix,...,production,virtual,data,infrastructure,environments,manager,performance,hp,security,-1.082731


In [148]:
target_key = "jdX_9214"
print (shuf_tok_bi_stp_traintest_cvjd_dict[target_key])

('Manager, Advertising Research | Comcast', ['comcast_spotlight', 'advertising', 'sales', 'division', 'comcast', 'cable', 'helps', 'power', 'cable', 'use', 'local', 'regional', 'national', 'advertisers', 'focused', 'providing', 'multi', 'platform', 'marketing', 'solutions', 'reach', 'audiences', 'effectively', 'efficiently', 'headquartered', 'new', 'york', 'offices', 'country', 'comcast_spotlight', 'presence', 'nearly', 'markets', 'approximately', 'million', 'owned', 'represented', 'subscribers', 'comcast_spotlight', 'offers', 'clients', 'easy', 'buy', 'easy', 'execute', 'options', 'customized', 'business', 'goals', 'clients', 'provide', 'customized', 'multi', 'screen', 'media', 'marketing', 'solutions', 'utilize', 'suite', 'products', 'way', 'best_suited', 'meet', 'objectives', 'media', 'solutions', 'provide', 'advertisers', 'reach', 'engage', 'connect', 'customers', 'viewing', 'content', 'device', 'screen', 'summary', 'comcast_spotlight', 'looking', 'enthusiastic', 'driven', 'intelle

In [112]:
ldaModel.show_topic(15,topn=30)

[('oracle', 0.08211039),
 ('database', 0.0664741),
 ('sql', 0.050024685),
 ('databases', 0.027454741),
 ('performance', 0.016716508),
 ('data', 0.016233724),
 ('server', 0.01537629),
 ('production', 0.012685963),
 ('tuning', 0.012318074),
 ('pl', 0.011750091),
 ('application', 0.010390515),
 ('backup', 0.008388655),
 ('development', 0.008360482),
 ('support', 0.0077947895),
 ('10g', 0.0077346545),
 ('dba', 0.007426074),
 ('11g', 0.0071635703),
 ('scripts', 0.0070453766),
 ('environment', 0.006596384),
 ('monitoring', 0.0058433604),
 ('rac', 0.0056936806),
 ('implemented', 0.0055624857),
 ('recovery', 0.005543728),
 ('maintenance', 0.005512437),
 ('environments', 0.005105917),
 ('rman', 0.004755905),
 ('unix', 0.0047557577),
 ('issues', 0.0046879062),
 ('replication', 0.0046422766),
 ('applications', 0.004472041)]

In [149]:
my_get_document_topics(ldaModel,mmcorpus_dict,'jdX_9214')

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,weight
0,team,business,skills,teams,technology,solutions,product,role,drive,help,0.310147
0,data,analysis,analytics,research,models,tools,reporting,statistical,machine_learning,business,0.254006
0,business,sales,product,market,financial,marketing,new,customer,revenue,strategy,0.145750
0,management,provide,support,ensure,manage,develop,business,processes,maintain,service,0.142928
0,marketing,design,com,digital,content,media,product,web,online,website,0.060688
0,health,care,staff,office,information,medical,state,department,services,duties,0.045673
0,project,management,managed,projects,program,team,process,developed,provided,budget,0.014609
0,training,web,created,content,developed,development,website,designed,students,learning,0.013893
0,software,development,mobile,code,android,embedded,design,developed,devices,hardware,0.008641
0,services,enterprise,development,solutions,infrastructure,technology,cloud,systems,architecture,management,0.000291


In [ ]:
my_get_term_topics(ldaModel,'web')

## Doc2vec

In [43]:
#print (common_texts)
if load_from_disk_doc2vec:
    doc2vecModel = Doc2Vec.load(load_doc2vecModel_path)
else:
    documents = [TaggedDocument(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[key][1], [key]) for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict]
    doc2vecModel = Doc2Vec(documents, vector_size=150, window=10, min_count=5, workers=4, dbow_words=1,dm=0,negative=5,epoch=20)
    doc2vecModel.save(save_doc2vecModel_path)  # you can continue training with the loaded model!
    # print (model.infer_vector(["system", "network","response"]))
    # print (model.infer_vector(["network","response","system"]))

In [ ]:
doc2vecModel.wv.most_similar('java')

In [ ]:
doc2vecModel.wv.most_similar('adobe_dreamweaver')

In [125]:
doc2vecModel.wv['documentation']


array([-0.2137294 ,  0.1337546 ,  0.32634223,  0.09490302,  0.3192235 ,
        0.05420782,  0.10385871, -0.01733131, -0.20321067,  0.13323388,
        0.23423141, -0.2365863 , -0.0378876 ,  0.04505159,  0.13834028,
        0.08021533,  0.12489429,  0.07572567,  0.03266483, -0.40802178,
       -0.3772196 , -0.18840459,  0.04078967, -0.13670379, -0.13872716,
        0.12110943, -0.21312757, -0.26661578, -0.08655941, -0.09003282,
        0.09439118, -0.02505813,  0.00346062,  0.00812404,  0.14034882,
       -0.07352781, -0.17872235, -0.15444279, -0.15304036,  0.0580807 ,
        0.08638655, -0.00209524,  0.22728871, -0.05840122, -0.08406451,
       -0.1940187 , -0.08516835, -0.03451778,  0.05916129, -0.05936216,
        0.01102082, -0.27514455, -0.32162926,  0.15717693,  0.04870271,
       -0.07865337,  0.25617862, -0.1282443 ,  0.3235245 , -0.28043726,
       -0.02033814,  0.0703186 , -0.21464944, -0.08402814,  0.12632047,
        0.0927387 , -0.00869285,  0.08387586, -0.0762924 , -0.23

In [ ]:
np.linalg.norm(doc2vecModel.wv['linear'])

# Get documents vectors

## TF-IDF doc_vecs

In [44]:
tfidf_doc_vecs_dict = {}
cnt = 0
for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict:
    vec = np.zeros(192069)
    for t in tfidfModel[mmcorpus_dict[key]]:
        vec[t[0]] = t[1]
    tfidf_doc_vecs_dict[key] = vec
    cnt += 1
    if cnt%1000 == 0:
        print (str(cnt/1000),"%")

#tfidf_doc_vecs_dict = {key: }for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict}


1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %
26.0 %
27.0 %
28.0 %
29.0 %
30.0 %
31.0 %
32.0 %
33.0 %
34.0 %
35.0 %
36.0 %
37.0 %
38.0 %
39.0 %
40.0 %
41.0 %
42.0 %
43.0 %
44.0 %
45.0 %
46.0 %
47.0 %
48.0 %
49.0 %
50.0 %
51.0 %
52.0 %
53.0 %
54.0 %
55.0 %
56.0 %
57.0 %
58.0 %
59.0 %
60.0 %
61.0 %
62.0 %
63.0 %
64.0 %
65.0 %
66.0 %
67.0 %
68.0 %
69.0 %
70.0 %
71.0 %
72.0 %
73.0 %
74.0 %
75.0 %
76.0 %
77.0 %
78.0 %
79.0 %
80.0 %
81.0 %
82.0 %
83.0 %
84.0 %
85.0 %
86.0 %
87.0 %
88.0 %
89.0 %
90.0 %
91.0 %
92.0 %
93.0 %
94.0 %
95.0 %
96.0 %
97.0 %
98.0 %
99.0 %
100.0 %
101.0 %
102.0 %
103.0 %
104.0 %
105.0 %


In [ ]:
print (dictionary[192069])

In [131]:
shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict['cvX_200']

('Senior Android Developer',
 ['amazon',
  'american',
  'electronic',
  'commerce',
  'cloud',
  'computing',
  'company',
  'based',
  'seattle',
  'washington',
  'team',
  'develop',
  'application',
  'track',
  'components',
  'involved',
  'manufacturing',
  'product',
  'scanning',
  'qr',
  'code',
  'product',
  'application',
  'provides',
  'user',
  'details',
  'participating',
  'product',
  'origins',
  'manufacturing',
  'date',
  'location',
  'item',
  'transparency',
  'label',
  'includes',
  'unique',
  'code',
  'track',
  'product',
  'information',
  'helps',
  'users',
  'understand',
  'authenticity',
  'product',
  'purchased',
  'app',
  'link',
  'https',
  'play',
  'google',
  'com',
  'store',
  'apps',
  'details',
  'id',
  'com',
  'amazon',
  'aba',
  'application',
  'hl',
  'en_us',
  'responsibilities',
  'team',
  'developed',
  'deployed',
  'amazon',
  'transparency',
  'application',
  'android',
  'platform',
  'team',
  'phases',
  'softwar

## LDA doc_vecs

In [132]:
lda_doc_vecs_dict = {key: my_get_lda_doc_vec(ldaModel,mmcorpus_dict,key,[])
                     for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict}

In [133]:
lda_doc_vecs_dict['cvX_200']

[0.016259778,
 4.2437267e-05,
 0.00016205877,
 9.0899324e-05,
 0.15269804,
 9.618233e-05,
 9.803099e-05,
 0.002230065,
 0.049748648,
 6.982723e-05,
 0.0061846026,
 0.00013964655,
 0.030891465,
 9.382833e-05,
 0.021692486,
 3.62868e-05,
 5.2786534e-05,
 8.8131404e-05,
 5.1799154e-05,
 0.14892858,
 7.3299445e-05,
 0.066003956,
 0.008239109,
 0.21188852,
 0.010806841,
 0.008535044,
 0.1685267,
 0.03237645,
 7.3349125e-05,
 0.06382117]

## Doc2vec doc_vecs

In [46]:
doc2vec_doc_vecs_dict = {key: doc2vecModel.docvecs[key] for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict}

In [47]:
print (doc2vecModel.docvecs['jdX_129'])
print (np.linalg.norm(doc2vec_doc_vecs_dict['jdX_129']))

[ 0.17788613 -0.09295878  0.03780525 -0.0485835   0.36987495 -0.10585401
 -0.3682367  -0.08746804  0.2108151  -0.04463128 -0.27002072  0.07084104
  0.0024846  -0.08104146 -0.15596811 -0.02696088 -0.00868823 -0.00717138
  0.2647378  -0.14178476  0.29395065 -0.41844562  0.52958083  0.43310592
  0.00560995  0.05329403 -0.13409038  0.03736943 -0.0584587  -0.165819
 -0.2583082   0.08178975 -0.20005804  0.04856459  0.10085849 -0.07338408
  0.03623416 -0.0806319  -0.08656701  0.2947452   0.03800932  0.06332364
  0.3632372   0.15346822 -0.02184591 -0.1101397  -0.21503086 -0.06965162
  0.23994385  0.01460334  0.00995536 -0.3949288   0.05998263  0.12484387
 -0.03580933 -0.06916828 -0.01720071  0.13923962  0.12063636 -0.11148477
  0.2504167  -0.20072377  0.4616316  -0.19655824 -0.3574043   0.1125458
  0.09558523  0.03734643 -0.18828465 -0.02633707 -0.2698268  -0.09720419
 -0.22431919  0.02808745 -0.07056797 -0.0919081   0.19625342  0.16721353
 -0.15429045  0.12868355  0.1820106   0.34397134  0.28

# Get job titles vectors

In [48]:
jts = [shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[key][0].split(" | ")[0] for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict]
tok_stp_bi_jts = my_tokenize(jts)


tokenized!
bigram model loaded!
bigram model applied!
stopword removed!


## LDA jt_vecs

In [49]:
# vectorize tokens
lda_tok_stp_bi_jts_tok_vecs = []
for tokens in tok_stp_bi_jts:
    tok_vecs = []
    for token in tokens:
        vec = np.zeros(30)
        try:
            for t in ldaModel.get_term_topics(dictionary.token2id[token],0.000):
                 vec[t[0]]=t[1]
        except:
            pass
        tok_vecs.append(vec)
    lda_tok_stp_bi_jts_tok_vecs.append(tok_vecs)

# append zero vectors to processed JT vectors with 0 components
for idx,val in enumerate(lda_tok_stp_bi_jts_tok_vecs):
    if len(val) == 0:
        lda_tok_stp_bi_jts_tok_vecs[idx] = [np.zeros(30)]
        print (idx)

# averaging token vectors and construct dict
lda_jts_avg_vecs = [np.average(tok_vecs, axis=0) for tok_vecs in lda_tok_stp_bi_jts_tok_vecs]
lda_jts_avg_vecs_dict = dict(zip(list(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict.keys()),lda_jts_avg_vecs))



26293
39542
48787
62588
65403
65450
69380


In [67]:
lda_jts_avg_vecs_dict['cvX_200']

array([7.34338983e-04, 3.96982886e-04, 1.00312141e-03, 1.03286836e-03,
       3.86042656e-04, 1.10769229e-03, 4.47694548e-04, 1.16252278e-03,
       1.31391410e-06, 2.26805326e-05, 1.09321991e-05, 2.67446992e-04,
       1.62559290e-04, 2.64719880e-05, 1.25019861e-03, 7.79424954e-04,
       2.57019807e-05, 9.13436059e-04, 6.20547026e-04, 5.24634857e-04,
       2.06644022e-04, 1.88406676e-04, 1.92428381e-04, 5.53995996e-03,
       1.17042796e-04, 3.16717493e-06, 5.35242394e-04, 1.53238002e-04,
       8.23113450e-04, 5.98960867e-04])

In [124]:
ldaModel.get_term_topics(dictionary.token2id['documentation'],0.000)

[(0, 0.0015049117),
 (1, 0.000979643),
 (2, 0.001344951),
 (3, 0.0028777758),
 (4, 0.004710251),
 (5, 2.0754582e-07),
 (7, 1.5796288e-05),
 (8, 0.0016033641),
 (9, 0.0033788644),
 (10, 0.0016462583),
 (11, 0.00435174),
 (12, 0.00014048217),
 (13, 0.0007048232),
 (14, 0.0072262185),
 (15, 0.000277608),
 (16, 0.0012997391),
 (17, 0.0031458682),
 (18, 0.0008188866),
 (19, 0.00026282744),
 (20, 1.567617e-05),
 (22, 0.001688449),
 (23, 3.123215e-05),
 (24, 0.00183755),
 (25, 0.0013150318),
 (26, 0.00022281766),
 (27, 0.0008595508),
 (29, 5.6834622e-05)]

## Doc2vec jt_vecs

In [50]:
def validate_wv(doc2vecModel,token):
    try:
        a = doc2vecModel.wv[token]
        return True
    except:
        return False

In [51]:
# vectorize tokens
doc2vec_tok_stp_bi_jts_tok_vecs = [[doc2vecModel.wv[token]
                                    for token in tokens if validate_wv(doc2vecModel,token)==True]
                                   for tokens in tok_stp_bi_jts]

# append zero vectors to processed JT vectors with 0 components
for idx,val in enumerate(doc2vec_tok_stp_bi_jts_tok_vecs):
    if len(val) == 0:
        doc2vec_tok_stp_bi_jts_tok_vecs[idx] = [np.zeros(150)]
        #print (idx)

# averaging token vectors and construct dict
doc2vec_jts_avg_vecs = [np.average(tok_vecs, axis=0) for tok_vecs in doc2vec_tok_stp_bi_jts_tok_vecs]
doc2vec_jts_avg_vecs_dict = dict(zip(list(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict.keys()),doc2vec_jts_avg_vecs))



In [ ]:
jts_avg_vecs_dict['cvY_3438']
shuf_tok_rmcm_bi_stp_traintest_cvjd_dict['cvY_3438']

In [ ]:
jt_idx = 902
print (jts[jt_idx])
print (tok_stp_bi_jts[jt_idx])

# Skill vectors (weighted by year)

## LDA skill_vecs

In [52]:
lda_skills_vecs_dict = {}
for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict:
    skills_vec = np.zeros(30)
    skill_denom = 0
    skills = shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[key][2]
    if skills!=None:
        for (skill_name,skill_year) in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[key][2]:
            skill_vec = np.zeros(30)
            try:
                for t in ldaModel.get_term_topics(dictionary.token2id[token],0.000):
                    skill_vec[t[0]]=t[1]
                skills_vec += skill_vec * skill_year
                skill_denom += skill_year
            except:
                pass
        #print (skills_vec,skill_denom)
    if (skill_denom != 0):
        lda_skills_vecs_dict[key] = skills_vec/skill_denom
    else:
        lda_skills_vecs_dict[key] = skills_vec
                
                

In [43]:
lda_skills_vecs_dict['cvX_200']

array([7.05398445e-04, 1.09679031e-03, 0.00000000e+00, 2.08002888e-03,
       4.54961089e-04, 6.74192212e-04, 3.47682736e-08, 7.12183304e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.44894035e-03, 2.26477883e-03,
       0.00000000e+00, 0.00000000e+00, 1.85739563e-03, 1.57037715e-03,
       7.88993759e-07, 2.80985900e-04, 0.00000000e+00, 3.03231180e-03,
       3.47495312e-04, 0.00000000e+00, 1.59568363e-03, 0.00000000e+00,
       4.53974371e-06, 1.79688260e-03])

## doc2vec skill_vecs

In [55]:
doc2vec_skills_vecs_dict = {}
for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict:
    skills_vec = np.zeros(150)
    skill_denom = 0
    skills = shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[key][2]
    if skills!=None:
        for (skill_name,skill_year) in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[key][2]:
            try:
                skills_vec += doc2vecModel.wv[skill_name] * skill_year
                skill_denom += skill_year
            except:
                pass
        #print (skills_vec,skill_denom)
    if (skill_denom != 0):
        doc2vec_skills_vecs_dict[key] = skills_vec/skill_denom
    else:
        doc2vec_skills_vecs_dict[key] = skills_vec
                
                

In [ ]:
doc2vec_skills_vecs_dict['cvX_200']

## Compute avg skill years for each cvX

In [122]:


avg_skills_years_dict = {}
for key in shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict:
    if key.startswith('cvX'):
        skills = shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[key][2]
        if skills!=None:
            no_skill = 0
            skill_year_sum = 0
            for (skill_name,skill_year) in skills:
                try:
                    skill_name_vec = doc2vecModel.wv[skill_name]
                    skill_year_sum += skill_year
                    no_skill += 1
                except:
                    pass
            try:
                avg_skills_years_dict[key] = skill_year_sum/no_skill
            except:
                avg_skills_years_dict[key] = 0

# Save vectors

In [134]:
with open(save_lda_doc_vecs_dict_path, 'wb') as f:
    pickle.dump(lda_doc_vecs_dict, f)

In [56]:
#save_tfidf_doc_vecs_dict_path = "./doc_vecs_dicts/tfidf_doc_vecs_dict.pickle"
save_lda_doc_vecs_dict_path = "../doc_vecs_dicts/lda_doc_vecs_dict.pickle"
save_doc2vec_doc_vecs_dict_path = "../doc_vecs_dicts/doc2vec_doc_vecs_dict.pickle"

save_lda_jts_avg_vecs_dict_path = "../doc_vecs_dicts/lda_jts_avg_vecs_dict.pickle"
save_doc2vec_jts_avg_vecs_dict_path = "../doc_vecs_dicts/doc2vec_jts_avg_vecs_dict.pickle"

save_lda_skills_vecs_dict_path = "../doc_vecs_dicts/lda_skills_vecs_dict.pickle"
save_doc2vec_skills_vecs_dict_path = "../doc_vecs_dicts/doc2vec_skills_vecs_dict.pickle"

save_avg_skills_years_dict_path = '../avg_skills_years_dict/avg_skills_years_dict.pickle'

with open(save_lda_doc_vecs_dict_path, 'wb') as f:
    pickle.dump(lda_doc_vecs_dict, f)
with open(save_doc2vec_doc_vecs_dict_path, 'wb') as f:
    pickle.dump(doc2vec_doc_vecs_dict, f)
print ("doc vec saved!")

with open(save_lda_jts_avg_vecs_dict_path, 'wb') as f:
    pickle.dump(lda_jts_avg_vecs_dict, f)
with open(save_doc2vec_jts_avg_vecs_dict_path, 'wb') as f:
    pickle.dump(doc2vec_jts_avg_vecs_dict, f)
print ("jt vec saved!")
    
with open(save_lda_skills_vecs_dict_path, 'wb') as f:
    pickle.dump(lda_skills_vecs_dict, f)
with open(save_doc2vec_skills_vecs_dict_path, 'wb') as f:
    pickle.dump(doc2vec_skills_vecs_dict, f)
print ("skills vec saved!")

with open(save_avg_skills_years_dict_path, 'wb') as f:
    pickle.dump(avg_skills_years_dict, f)
print ("avg_skills_years_dict saved!")

doc vec saved!
jt vec saved!
skills vec saved!
avg_skills_years_dict saved!


In [123]:

with open(save_avg_skills_years_dict_path, 'wb') as f:
    pickle.dump(avg_skills_years_dict, f)
print ("avg_skills_years_dict saved!")

avg_skills_years_dict saved!


In [52]:
with open(save_lda_doc_vecs_dict_path, 'wb') as f:
    pickle.dump(lda_doc_vecs_dict, f)

# Load vectors

In [3]:
#save_tfidf_doc_vecs_dict_path = "./doc_vecs_dicts/tfidf_doc_vecs_dict.pickle"
load_lda_doc_vecs_dict_path = "../doc_vecs_dicts/lda_doc_vecs_dict.pickle"
load_doc2vec_doc_vecs_dict_path = "../doc_vecs_dicts/doc2vec_doc_vecs_dict.pickle"

load_lda_jts_avg_vecs_dict_path = "../doc_vecs_dicts/lda_jts_avg_vecs_dict.pickle"
load_doc2vec_jts_avg_vecs_dict_path = "../doc_vecs_dicts/doc2vec_jts_avg_vecs_dict.pickle"

load_lda_skills_vecs_dict_path = "../doc_vecs_dicts/lda_skills_vecs_dict.pickle"
load_doc2vec_skills_vecs_dict_path = "../doc_vecs_dicts/doc2vec_skills_vecs_dict.pickle"

load_avg_skills_years_dict_path = '../avg_skills_years_dict/avg_skills_years_dict.pickle'

with open(load_lda_doc_vecs_dict_path, 'rb') as f:
    lda_doc_vecs_dict = pickle.load(f)
with open(load_doc2vec_doc_vecs_dict_path, 'rb') as f:
    doc2vec_doc_vecs_dict = pickle.load(f)
print ("doc vec loaded!")

with open(load_lda_jts_avg_vecs_dict_path, 'rb') as f:
    lda_jts_avg_vecs_dict = pickle.load(f)
with open(load_doc2vec_jts_avg_vecs_dict_path, 'rb') as f:
    doc2vec_jts_avg_vecs_dict = pickle.load(f)
print ("jt vec loaded!")
    
with open(load_lda_skills_vecs_dict_path, 'rb') as f:
    lda_skills_vecs_dict = pickle.load(f)
with open(load_doc2vec_skills_vecs_dict_path, 'rb') as f:
    doc2vec_skills_vecs_dict = pickle.load(f)
print ("skills vec loaded!")


with open(load_avg_skills_years_dict_path, 'rb') as f:
    avg_skills_years_dict = pickle.load(f)
print ("avg_skills_years_dict loaded!")

doc vec loaded!
jt vec loaded!
skills vec loaded!
avg_skills_years_dict loaded!


# Plot vectors

In [16]:
def pca_vis(doc_vecs_dict):
    #PCA(n_components=None, copy=True, whiten=False, svd_solver=’auto’, tol=0.0, iterated_power=’auto’, random_state=None)
    tags = list(doc_vecs_dict.keys())
    doc_vecs = list(doc_vecs_dict.values())
    
    data = []
    X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
    pca = PCA(n_components=2)
    Y = pca.fit_transform(doc_vecs)
   # print (Y)
    print(pca.explained_variance_ratio_)  
#     print(pca.singular_values_) 
#     print (pca.components_)
    
    cnt_data = 0
    cvX_no = 10
    cvY_no = 10
    jdX_no = 10
    for idx,tag in enumerate(tags):
        if cnt_data>=50:
            break
        if (tag in doc_vecs_dict):
            if(tag.startswith("jd") and jdX_no > 0):
                color = 'rgba(30, 128, 256, 0.9)'
                #jdX_no -= 1
            elif (tag.startswith("cvX") and cvX_no > 0):
                color = 'rgba(200, 60, 60, 0.7)'
                #cvX_no -= 1
            elif (tag.startswith("cvY") and cvY_no > 0):
                color = 'rgba(60, 200, 60, 0.7)'
                #cvY_no -= 1
            trace = dict(
                type='scatter',
                x=[Y[idx][0]],
                y=[Y[idx][1]],
                mode='markers+text',
                text=shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[tag][0],
                textfont=dict(
                   size=10
                ),
                textposition='bottom center',
                #name=(" <br> ").join(textwrap.wrap(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[tag][3],width=200)),
                marker=dict(
                    color=color,
                    size=7,
                    line=dict(
                        color='rgba(217, 217, 217, 0.9)',
                        width=0.3),
                    opacity=1.0),
                hoverlabel=dict(namelength=-1),
            )
            data.append(trace)
        else:
            print (tag)
        cnt_data += 1 
        if cnt_data%100==0:
            print (cnt_data)

    layout = dict(
        showlegend=False,
        scene=dict(
            xaxis=dict(title='PC1'),
            yaxis=dict(title='PC2')
        )
    )

    fig = dict(data=data, layout=layout)
    return fig

In [153]:
def pca_vis_skills(doc_vecs_dict):
    #PCA(n_components=None, copy=True, whiten=False, svd_solver=’auto’, tol=0.0, iterated_power=’auto’, random_state=None)
    tags = list(doc_vecs_dict.keys())
    doc_vecs = list(doc_vecs_dict.values())
    #pprint (doc_vecs)
    data = []
    X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
    pca = PCA(n_components=2)
    Y = pca.fit_transform(doc_vecs)
    print (Y)
    print(pca.explained_variance_ratio_)  
    print(pca.singular_values_) 
    print (pca.components_) 
    cnt_data = 0

    for idx,tag in enumerate(tags):
        if cnt_data>=100:
            break
        else:
            trace = dict(
                type='scatter',
                x=[Y[idx][0]],
                y=[Y[idx][1]],
                mode='markers+text',
                text=tag,
                textfont=dict(
                   size=12
                ),
                textposition='bottom center',
                #name=(" <br> ").join(textwrap.wrap(shuf_del_tok_rmcm_bi_stp_traintest_cvjd_dict[tag][3],width=200)),
                marker=dict(
                    color='rgba(30, 128, 256, 0.9)',
                    size=7,
                    line=dict(
                        color='rgba(217, 217, 217, 0.14)',
                        width=0.3),
                    opacity=0.8),
                hoverlabel=dict(namelength=-1),
            )
            data.append(trace)
            print (tag)
        cnt_data += 1 
        if cnt_data%100==0:
            print (cnt_data)

    layout = dict(
        showlegend=False,
        scene=dict(
            xaxis=dict(title='PC1'),
            yaxis=dict(title='PC2')
        )
    )
    #print (data)
    fig = dict(data=data, layout=layout)
    return fig

In [152]:
plot(pca_vis(doc2vec_doc_vecs_dict), filename='./plot/doc2vec150_rmcmstp2xall.html')


[[-0.24260472 -0.20879987]
 [-0.71942586 -0.40485791]
 [-0.89278511 -0.92079885]
 [ 1.36013207 -0.52260146]
 [-1.195264   -0.05570141]
 [-0.75375567 -0.3547559 ]
 [-0.37354955 -0.23905566]
 [ 2.8160612  -0.64634058]
 [ 0.4182495   2.80379058]
 [-0.41705788  0.54912105]]
[0.24355084 0.18026978]
[3.69308389 3.17728246]
[[-0.01529612 -0.00673311 -0.07554485  0.00062055  0.19584139 -0.07877857
   0.12749464  0.061733   -0.06275134  0.09852203  0.00665918  0.12806783
   0.01837928 -0.00437685  0.04600716 -0.08648468 -0.14795692  0.13357141
   0.01678662 -0.08935961 -0.06050685 -0.04994112 -0.09353097  0.01202057
  -0.03998899 -0.04219926  0.12288799  0.06181452 -0.0188267  -0.11449381
  -0.08686037 -0.06389095 -0.04631225 -0.13883738 -0.04590409  0.01981794
   0.06839022  0.09555577 -0.11694236  0.11175733 -0.09307408 -0.07963632
  -0.0038626   0.11308908  0.06437709  0.00642054 -0.08394615 -0.04067926
  -0.03777022 -0.06912345  0.0760468  -0.0849968   0.07402177 -0.06749016
   0.09061325  

FileNotFoundError: [Errno 2] No such file or directory: './plot/doc2vec150_rmcmstp2xall.html'

In [18]:
plot(pca_vis(lda_jts_avg_vecs_dict), filename='../plot/lda_jt2vecx50.html')

[0.28900595 0.17795095]


KeyboardInterrupt: 

In [19]:
plot(pca_vis(doc2vec_jts_avg_vecs_dict), filename='../plot/doc2vec_jt2vecx50.html')


[0.10597373 0.08551381]


KeyboardInterrupt: 

In [15]:
plot(pca_vis(lda_doc_vecs_dict), filename='./plot/lda30stp2xall.html')

[[-0.05039958  0.03859506]
 [-0.03419566  0.0861269 ]
 [-0.07366926  0.27994091]
 ...
 [-0.07312264  0.40725647]
 [-0.24917448 -0.20084418]
 [-0.00829532 -0.1973198 ]]
[0.08863026 0.0832842 ]
[47.96617329 46.49704844]
[[ 4.81831878e-01 -1.91317915e-02 -1.73822940e-01 -6.77389678e-02
   1.12457993e-02  6.08814526e-01  6.17651312e-02  6.54035775e-02
  -1.83905257e-01 -1.49501617e-01 -5.79841613e-02  4.48556686e-02
  -6.19946742e-02 -1.25460134e-01 -1.08613386e-02 -1.77772481e-02
  -8.37634121e-02  6.53536539e-03 -2.50078171e-02  2.70903850e-03
   8.04243629e-02  3.65390238e-02 -5.01597605e-01  4.42937085e-02
  -8.00645639e-03  3.19003008e-02  6.23287989e-06  1.48183023e-02
  -2.02371291e-02  3.82537620e-02]
 [-2.35400004e-01 -1.44610846e-02  7.53951878e-01  6.16894230e-02
   1.93335609e-01 -4.17900624e-02  2.42343115e-01  9.84391121e-02
  -2.48386041e-02 -1.36169920e-01 -6.66784137e-02  4.58026685e-02
  -9.18900647e-03 -7.62530218e-02  6.38828812e-04 -1.91883643e-02
  -8.97132078e-02 -1.

KeyboardInterrupt: 

In [86]:
skills_plot = ['sql', 'java', 'linux', 'javascript', 'microsoft_office', 'excel', 'oracle', 'active_directory', 'project_management', 'html', 'customer_service', 'python', 'cisco', 'c#', 'vmware', 'database', 'css', 'security', 'sql_server', '.net', 'help_desk', 'desktop_support', 'unix', 'sharepoint', 'mysql', 'training', 'testing', 'jquery', 'powerpoint', 'networking', 'jira', 'xml', 'ms_office', 'c++', 'aws', 'system_administrator', 'c', 'git', 'visio', 'word', 'asp', 'data_analysis', 'ms_sql_server', 'php', 'management', 'tech_support', 'business_intelligence', 'sap', 'agile', 'outlook', 'marketing', 'data_entry', 'access', 'service_desk', 'operations', 'sales', 'change_management', 'mac', 'exchange', 'technical_support', 'etl', 'asp.net', 'ajax', 'photoshop', 'business_analysis', 'program_management', 'databases', 'business_development', 'android', 'strategic_planning', 'j2ee', 'quality_assurance', 'information_technology', 'pl/sql', 'angular', 'windows', 'tableau', 'call_center', 'dns', 'eclipse', 'sdlc', 'salesforce', 'integration', 'process_improvement', 'software_development', 'microsoft_sharepoint', 'maintenance', 'documentation', 'vpn', 'lan', 'itil', 'budget', 'scrum', 'remedy', 'microsoft_project', 'visual_studio', 'db2', 'scheduling', 'crm', 'pmp', 'qa', 'information_security', 'spring', 'architecture', 'bootstrap', 'mvc', 'windows_7', 'ios', 'hadoop', 'wordpress', 'jsp', 'apache', 'scrum_master', 'customer_support', 'html_5', 'html5', 'dhcp', 'citrix', 'bgp', 'json', 'system_admin', 'microsoft_excel', 'helpdesk_support', 'solaris', 'perl', 'microsoft_word', 'account_management', 'ui', 'retail_sales', 'hibernate', 'web_services', 'angularjs', 'receptionist', 'network_security', 'informatica', 'troubleshooting', 'api', 'vendor_management', 'comptia', 'jenkins', 'firewalls', 'net', 'scripting', 'network_administration', 'risk_management', 'pm', 'graphic_design', 'tcp/ip', 'rest', 'css3', 'cobol', 'sccm', 'team_building', 'deployment', 'data_center', 'waterfall', 'integrator', 'devops', 'selenium', 'product_management', 'visual_basic', 'inventory', 'agile_scrum', 'peoplesoft', 'web_design', 'customer_relationship_management', 'dba', 'budgeting', 'aix', 'ospf', 'bi', 'powershell', 'erp', 'system_administration', 'voip', 'illustrator', 'adobe', 'soap', 'logistics', 'react', 'cloud_computing', 'technical_writing', 'ms_project', 'microsoft_windows', 'wireless', 'system_analyst', 'adobe_photoshop', 'disaster_recovery', 'ccna', 'product_development', 'problem_solving', 'cyber_security', 'database_management', 'virtualization', 'ssrs', 'ms_access', 'matlab', 'accounting', 'leadership_development', 'sas']
skills_vecs_plot = {}
for key in skills_plot:
    try:
        skills_vecs_plot[key] = doc2vecModel.wv[key]
    except:
        pass
print (len(skills_vecs_plot))
plot(pca_vis_skills(skills_vecs_plot), filename='../plot/doc2vec_skillx200.html')


133
[[ 2.62013997e-01 -6.03224137e-01]
 [ 9.01724653e-01 -8.12290542e-02]
 [-3.57592286e-01  2.47007990e-01]
 [ 1.69106184e+00  4.08736791e-01]
 [-1.02255436e-01 -7.51214129e-01]
 [-6.77448550e-02 -7.93887181e-01]
 [-8.72979926e-01  8.07022531e-01]
 [ 1.49070786e+00  4.10387870e-01]
 [ 3.81405747e-01 -4.13532850e-02]
 [-1.28977027e+00  1.13123977e+00]
 [-9.30756083e-01  6.36625291e-01]
 [-1.59144005e-01 -4.47847146e-01]
 [ 1.71950096e+00  6.34335045e-01]
 [-7.60389063e-01  1.28839754e-01]
 [-1.04359903e+00 -1.85893105e-02]
 [-3.04961867e-01 -3.00946886e-01]
 [-5.34974984e-03 -2.98144249e-01]
 [ 6.17888431e-01 -2.05702677e-02]
 [-7.62817799e-01 -4.53598496e-01]
 [-2.28850412e-01 -3.23041132e-01]
 [ 2.12254425e+00  6.16256824e-01]
 [-2.95820188e-01 -7.01403904e-01]
 [-9.89755600e-01  6.73267026e-01]
 [ 5.36241937e-01 -4.19234699e-01]
 [ 1.24471163e+00  6.74770979e-03]
 [ 9.74698640e-02  1.67369551e-01]
 [ 9.82514653e-01  7.66407039e-03]
 [-2.08182463e-03 -3.48133838e-01]
 [-2.51391274e-0

KeyboardInterrupt: 

In [100]:
skills_plot = ['sql', 'java', 'linux', 'javascript', 'microsoft_office', 'excel', 'oracle', 'active_directory', 'project_management', 'html', 'customer_service', 'python', 'cisco', 'c#', 'vmware', 'database', 'css', 'security', 'sql_server', '.net', 'help_desk', 'desktop_support', 'unix', 'sharepoint', 'mysql', 'training', 'testing', 'jquery', 'powerpoint', 'networking', 'jira', 'xml', 'ms_office', 'c++', 'aws', 'system_administrator', 'c', 'git', 'visio', 'word', 'asp', 'data_analysis', 'ms_sql_server', 'php', 'management', 'tech_support', 'business_intelligence', 'sap', 'agile', 'outlook', 'marketing', 'data_entry', 'access', 'service_desk', 'operations', 'sales', 'change_management', 'mac', 'exchange', 'technical_support', 'etl', 'asp.net', 'ajax', 'photoshop', 'business_analysis', 'program_management', 'databases', 'business_development', 'android', 'strategic_planning', 'j2ee', 'quality_assurance', 'information_technology', 'pl/sql', 'angular', 'windows', 'tableau', 'call_center', 'dns', 'eclipse', 'sdlc', 'salesforce', 'integration', 'process_improvement', 'software_development', 'microsoft_sharepoint', 'maintenance', 'documentation', 'vpn', 'lan', 'itil', 'budget', 'scrum', 'remedy', 'microsoft_project', 'visual_studio', 'db2', 'scheduling', 'crm', 'pmp', 'qa', 'information_security', 'spring', 'architecture', 'bootstrap', 'mvc', 'windows_7', 'ios', 'hadoop', 'wordpress', 'jsp', 'apache', 'scrum_master', 'customer_support', 'html_5', 'html5', 'dhcp', 'citrix', 'bgp', 'json', 'system_admin', 'microsoft_excel', 'helpdesk_support', 'solaris', 'perl', 'microsoft_word', 'account_management', 'ui', 'retail_sales', 'hibernate', 'web_services', 'angularjs', 'receptionist', 'network_security', 'informatica', 'troubleshooting', 'api', 'vendor_management', 'comptia', 'jenkins', 'firewalls', 'net', 'scripting', 'network_administration', 'risk_management', 'pm', 'graphic_design', 'tcp/ip', 'rest', 'css3', 'cobol', 'sccm', 'team_building', 'deployment', 'data_center', 'waterfall', 'integrator', 'devops', 'selenium', 'product_management', 'visual_basic', 'inventory', 'agile_scrum', 'peoplesoft', 'web_design', 'customer_relationship_management', 'dba', 'budgeting', 'aix', 'ospf', 'bi', 'powershell', 'erp', 'system_administration', 'voip', 'illustrator', 'adobe', 'soap', 'logistics', 'react', 'cloud_computing', 'technical_writing', 'ms_project', 'microsoft_windows', 'wireless', 'system_analyst', 'adobe_photoshop', 'disaster_recovery', 'ccna', 'product_development', 'problem_solving', 'cyber_security', 'database_management', 'virtualization', 'ssrs', 'ms_access', 'matlab', 'accounting', 'leadership_development', 'sas']
skills_vecs_plot = {}
for key in skills_plot:
    try:
        #print (key)
        skills_vecs = np.zeros(30)
        for t in ldaModel.get_term_topics(dictionary.token2id[key],0.000):
            #print (t)
            skills_vecs[t[0]]=t[1]
        skills_vecs_plot[key] = list(skills_vecs)
    except:
        pass
pprint (len(skills_vecs_plot))
plot(pca_vis_skills(skills_vecs_plot), filename='../plot/lda_skillx200.html')


134
[[ 5.38964159e-02  2.56656041e-03]
 [ 1.01417391e-04  3.06369156e-03]
 [ 2.40531532e-05 -7.14548388e-04]
 [-6.37437645e-04  2.66830858e-04]
 [-1.25228552e-03 -3.76603166e-04]
 [ 7.92746736e-02 -4.90170263e-03]
 [-2.09857074e-03 -1.14623007e-03]
 [-1.12570957e-03 -2.44854240e-05]
 [-2.63040530e-03 -1.04091364e-03]
 [-3.42840617e-03 -9.43939094e-04]
 [-2.35000452e-03 -1.14886910e-03]
 [ 6.44954574e-02 -1.81947274e-03]
 [-1.30890148e-03 -2.95101609e-04]
 [ 2.45230219e-03  1.74096867e-03]
 [-2.50512485e-03 -1.16804903e-03]
 [ 3.33662600e-03  8.35830280e-04]
 [ 2.05098469e-03 -8.41950892e-04]
 [-7.58039721e-04 -1.11601389e-03]
 [-2.23272347e-03  3.70468658e-05]
 [ 3.03695508e-03  7.47566771e-02]
 [-1.34238419e-03 -3.37767152e-04]
 [-2.44696012e-03 -1.22326389e-03]
 [-2.60359856e-03 -9.89933303e-04]
 [-2.29701109e-03  1.97103675e-03]
 [-8.50518404e-04  1.12039167e-03]
 [-2.94432956e-03 -8.38788814e-04]
 [-2.44316598e-03 -9.80930892e-04]
 [-2.13511345e-03 -8.74787137e-04]
 [-2.32091963e-0

KeyboardInterrupt: 

In [116]:
ldaModel.get_term_topics(dictionary.token2id['oracle'],0.000)

[(1, 2.6746276e-08),
 (2, 3.5863147e-06),
 (3, 0.009783154),
 (4, 0.00031256033),
 (5, 1.575734e-08),
 (6, 6.049183e-05),
 (7, 0.0009873048),
 (11, 1.0292588e-06),
 (15, 0.08145437),
 (16, 0.0017410968),
 (18, 0.007364619),
 (19, 0.008160664),
 (22, 1.7669049e-07),
 (24, 0.0008030556),
 (26, 0.00022078575),
 (28, 3.6528972e-06),
 (29, 5.455884e-05)]

In [117]:
skills_vecs = np.zeros(30)
for t in ldaModel.get_term_topics(dictionary.token2id['python'],0.000):
    skills_vecs[t[0]]=t[1]
np.linalg.norm(skills_vecs)

0.017707806471331355

In [ ]:
np.linalg.norm(skills_vecs)

In [121]:
doc2vecModel.wv['python']
np.linalg.norm(doc2vecModel.wv['sas'])

3.291287

In [120]:
doc2vecModel.wv['python']
np.linalg.norm(doc2vecModel.wv['oracle'])

2.4499638

Todo:
1. systemmetic visualize (axis represents different attribute)
2. evaluation
3. 3 + 1 metircs (jts skills jds)
4. add education
5. comprehensive but simple
